Neural Network Model

In [ ]:
#Import the packages that are used
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras.backend as K
from sklearn.metrics import r2_score

# Custom rounding layer that turns the optimum revolution prediction from a float value to a integer
class RoundLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(RoundLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        super(RoundLayer, self).build(input_shape)

    def call(self, x):
        x_rounded = K.round(x)
        return x_rounded

    def compute_output_shape(self, input_shape):
        return input_shape

# Converts the CSV File into the data matrix
data = pd.read_csv("C:/Users/xwint/lambert_data (2).csv")

# Visualize the data using Seaborn's pairplot
sns.pairplot(data)
plt.show()

# Split the data into input (X) and output (y) variables from the data matriax
X = data.iloc[:, :7].values
y = data.iloc[:, 7:].values

# Split the data into training and testing sets at a 80:20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the input data using MinMaxScaler: Allows the machine model to create an accurate fitting without much scaling errors
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network model with L2 regularization, dropout, and the custom rounding layer
#The Dropout and L2 are used to prevent overfitting
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), input_shape=(7,)),
    tf.keras.layers.Dropout(0.05),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.05),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.05),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.05),
    tf.keras.layers.Dense(7)
])

# Add custom rounding layer for the revolution output
model.add(tf.keras.layers.Lambda(lambda x: K.concatenate([x[:, :6], RoundLayer()(x[:, 6:])], axis=-1)))

# Compile the model using an adam optimizer and mean square error loss function
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model with 200 epochs with a batch size of 16.
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=16)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Mean Absolute Error: {mae}")

# Plot the training history to visualize its performance.
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.show()

# Make predictions on the test set 
y_pred = model.predict(X_test)

# Plot predicted values vs. testing values
fig, ax = plt.subplots(1, 7, figsize=(20, 5))

for i in range(7):
    ax[i].scatter(y_test[:, i], y_pred[:, i], label='Predicted vs. Actual')

    #Include a red dotted line with slope 1 to show the ideal values
    ax[i].plot([y_test[:, i].min(), y_test[:, i].max()], [y_test[:, i].min(), y_test[:, i].max()], 'r--')
    ax[i].set_xlabel('Actual')
    ax[i].set_ylabel('Predicted')
    ax[i].set_title(f"Output {i + 1}")

plt.tight_layout()
plt.show()

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print('R2 score: {:.2f}'.format(r2))

Results and Display

In [ ]:
import seaborn as sns

# Calculate absolute error for each predicted value
abs_error = np.abs(y_test - y_pred)

# Plot predicted values vs. testing values with absolute error
fig, ax = plt.subplots(1, 6, figsize=(20, 5))

#Create the heatmaps for visualization for the 6 velocity components. Compare the predicted and actual result.
for i in range(6):
    scatter = ax[i].scatter(y_test[:, i], y_pred[:, i], c=abs_error[:, i], cmap='coolwarm', label='Predicted vs. Actual')
    ax[i].plot([y_test[:, i].min(), y_test[:, i].max()], [y_test[:, i].min(), y_test[:, i].max()], 'r--')
    ax[i].set_xlabel('Actual')
    ax[i].set_ylabel('Predicted')
    ax[i].set_title(f"Velocity Component {i + 1}")

plt.tight_layout()
plt.colorbar(scatter, ax=ax, label='Absolute Error')
plt.show()

# Create a heatmap/confusion matrix for the predicted revolutions against the correct test labels
pred_vs_test_df = pd.DataFrame({'Predicted Revolution': y_pred[:, 6], 'Actual Revolution': y_test[:, 6]})
heatmap_data = pred_vs_test_df.groupby(['Predicted Revolution', 'Actual Revolution']).size().reset_index(name='Count')
heatmap_data = heatmap_data.pivot_table(values='Count', index='Predicted Revolution', columns='Actual Revolution', fill_value=0)

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='coolwarm')
plt.title('Predicted vs. Actual Revolutions Heatmap for Optimal Revolutions')
plt.xlabel('Actual Revolution')
plt.ylabel('Predicted Revolution')
plt.show()

from sklearn.metrics import r2_score

# Assuming you already have y_test (actual values) and y_pred (predicted values)

# Ensure y_test and y_pred have the same dimensions
assert y_test.shape == y_pred.shape, f"y_test and y_pred have different shapes: {y_test.shape} vs {y_pred.shape}"

# Calculate the R2 score for each predicted feature
num_features = y_test.shape[1]
for i in range(num_features):
    r2 = r2_score(y_test[:, i], y_pred[:, i])
    print(f"R2 score for feature {i+1}: {r2:.4f}")